## Gender Classification Project

## 1.google drive permision check

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2.Import zip file from drive

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/Comys_Hackathon5.zip"
extract_path = "/content/Comys_Hackathon5"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

## 3.Import the libraries

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms,models,datasets
from torchvision.transforms import ToTensor, Normalize, Resize, Compose, RandomHorizontalFlip, ColorJitter
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

## 4.HyperParaMeters

In [ ]:
BATCH_SIZE = 32
LR = 1e-4
EPOCHS = 5
IMAGE_SIZE = 224

## 5.Data Transforms

In [ ]:
train_transform=transforms.Compose([
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_transform=transforms.Compose([
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

##6. Find The Directories Of Train and Validation Datasets

In [ ]:
!find /content/Comys_Hackathon5/ -name "train"
!find /content/Comys_Hackathon5/ -name "val"

/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/train
/content/Comys_Hackathon5/Comys_Hackathon5/Task_B/train
/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/val
/content/Comys_Hackathon5/Comys_Hackathon5/Task_B/val


## 7.breakdown into train and test datasets

In [ ]:
train_dataset = datasets.ImageFolder("/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/train", transform=train_transform)
val_dataset = datasets.ImageFolder("/content/Comys_Hackathon5/Comys_Hackathon5/Task_A/val", transform=val_transform)

## 8. length of training dataset

In [ ]:
len(train_dataset)

1926

## 9. assigned male as 1 and female as 0

In [ ]:
from collections import Counter

targets = [label for _, label in train_dataset]
class_counts = Counter(targets)

print(class_counts)


Counter({1: 1623, 0: 303})


## 10. Create Weights For each Sample

In [ ]:
import numpy as np
from torch.utils.data import WeightedRandomSampler

class_sample_counts = [class_counts[0], class_counts[1]]
weights = 1. / np.array(class_sample_counts)

sample_weights = [weights[label] for label in targets]


## 11.initialize the weightRandomSampler

In [ ]:
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)


## 12. load the DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler,num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## 13. use the gpu

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## 14. use of pretrained Model (RESNET18)

In [ ]:
model = models.resnet18(pretrained=True)
#dropout
model.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model.fc.in_features, 2)
)

model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 148MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 14. Optimizaor and criterion

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

## 15 .Training the dataset

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

for epoch in range(EPOCHS):
    # -------------------- Training --------------------
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    train_losses.append(train_loss / total)
    train_accuracies.append(train_acc)

    # -------------------- Validation --------------------
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = 100 * correct / total
    val_losses.append(val_loss / total)
    val_accuracies.append(val_acc)

    # -------------------- Print Epoch Summary --------------------
    print(f"Epoch {epoch+1}/{EPOCHS}: "
          f"Train Loss={train_losses[-1]:.4f}, Val Loss={val_losses[-1]:.4f}, "
          f"Train Acc={train_acc:.2f}%, Val Acc={val_acc:.2f}%")


Epoch 1/10: Train Loss=0.0290, Val Loss=0.3439, Train Acc=99.17%, Val Acc=91.47%
Epoch 2/10: Train Loss=0.0090, Val Loss=0.3869, Train Acc=99.64%, Val Acc=91.47%
Epoch 3/10: Train Loss=0.0104, Val Loss=0.3794, Train Acc=99.74%, Val Acc=91.71%
Epoch 4/10: Train Loss=0.0036, Val Loss=0.3573, Train Acc=99.95%, Val Acc=92.65%
Epoch 5/10: Train Loss=0.0011, Val Loss=0.3745, Train Acc=100.00%, Val Acc=92.42%
Epoch 6/10: Train Loss=0.0016, Val Loss=0.4732, Train Acc=100.00%, Val Acc=91.47%
Epoch 7/10: Train Loss=0.0138, Val Loss=0.3664, Train Acc=99.64%, Val Acc=91.94%
Epoch 8/10: Train Loss=0.0058, Val Loss=0.3507, Train Acc=99.84%, Val Acc=91.94%
Epoch 9/10: Train Loss=0.0209, Val Loss=0.4119, Train Acc=99.22%, Val Acc=91.00%
Epoch 10/10: Train Loss=0.0115, Val Loss=0.4281, Train Acc=99.53%, Val Acc=92.18%


## 16.Classification Matrix

In [ ]:
print("Final Validation Metrics (Task A - Gender Classification):")
print(classification_report(all_labels, all_preds, target_names=train_dataset.classes))

Final Validation Metrics (Task A - Gender Classification):
              precision    recall  f1-score   support

      female       0.78      0.85      0.81        79
        male       0.96      0.94      0.95       343

    accuracy                           0.93       422
   macro avg       0.87      0.90      0.88       422
weighted avg       0.93      0.93      0.93       422



## 17. Accuracy Matrices

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy * 100:.2f}%")## VAlidation Accuracy

Accuracy: 92.18%
